# Sequence hacking
multidimensional expansion to the vector class created in ch9. Will implement methods that allow the object to behave (quack) like a sequence. For this, the object must define 2 methods:
- \_\_len__: get the size of the object
- \_\_getitem__: get an item at a given index (slicing)



## Discussion: variable args in constructor
a Vector class could receive variable size constructors through an *args in the \_\_init__ fn and it would work, but the best practice for a sequence constructor is to take the data as an iterable argument in the constructor (a list, generator, object with \_\_iter__method, etc.)

In [19]:
import math
from array import array
import reprlib #repr that truncates after certain length

class Vector:
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1] # removes array('d' and trailing )
        class_name = type(self).__name__
        return '{}({})'.format(class_name, components)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)
    
    # Sequence protocol
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, position):
        return self._components[position]


In [22]:
# Sequence protocol
v = Vector([1,5,2,5,32,5])
print(len(v))
print(v[2])
print(v[0:2])

6
2.0
array('d', [1.0, 5.0])


In [41]:
# Understanding the slice objects
class MySeq:
    """ Just returns the index object to see how it's treated by the getitem method"""
    def __getitem__(self, index):
        return index
    
s = MySeq()
print(s[1:4])
# A [1:4] slice is turned into a slice object where slice(start, stop, stride)
print(s[1:4:2, 5:10:5])
# Actually it's a tuple of slice objects
print(dir(slice))
# Slice has an indices method, which produces normalized tuples of nonnegative
# start, stop and stride integers adjusted to fit within the given sequence
print(help(slice.indices))
print(slice(None, 10, 2).indices(5), "->'ABCDE'[0:5:2] == 'ABCDE'[:10:2]")
'ABCDE'[0:5:2] == 'ABCDE'[:10:2]

print(slice(-3, None, None).indices(5), "->'ABCDE'[-3:] == 'ABCDE'[2:5:1]")
'ABCDE'[-3:] == 'ABCDE'[2:5:1]

slice(1, 4, None)
(slice(1, 4, 2), slice(5, 10, 5))
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.

None
(0, 5, 2) ->'ABCDE'[0:5:2] == 'ABCDE'[:10:2]
(2, 5, 1) ->'ABCDE'[-3:] == 'ABCDE'[2:5:1]


True

In [57]:
# The problem with the current getitem method in vector is that a slice returns
# an object of another type (an array) when it would be better if it 
# returned another object of the same type (vector). 
# The handling of the slice is delegated to _components, which is an array and already knows how to do it
class VectorV2(Vector):
    def __getitem__(self, index):
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, int): # could use numbers.Integral instead of int
            return self._components[index]
        else:
            msg = f"{cls.__name__} indices must be integers"
            raise TypeError(msg)

In [55]:
v1 = Vector([23,35,6,3,3,2,46,3,2,4])
v2 = VectorV2([23,35,6,3,3,2,46,3,2,4])

v2[1:3]
#v1[1:3]

VectorV2([35.0, 6.0])

## Dynamic attribute access
Vector2d allowed to access attributes by name (v.x, v.y), while this object doesn't define names.

The method \_\_getattr__ provides a way to do it. it is invoked by the interpreter when an attribute lookup fails. If I write v.x, the following happends in order:
1. checks if the v instance has an attribute named v
2. go to the class
3. then up the inheritance graph
4. finally if x is never found, the getattr method is called

In [66]:
class VectorV3(VectorV2):
    shortcut_names = 'xyzt'
    
    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name) # returns index of the given name in the xyzt sequence, -1 if not found
            if 0 <= pos < len(self._components):
                return self._components[pos]
            else:
                msg = f"{cls.__name__} has no attribute {name}"
                raise AttributeError(msg)
            


In [68]:
v = VectorV3([2,4,5,2,5,2,5])
print(v.x)
v.k

2.0


AttributeError: VectorV3 has no attribute k

# Set attributes
The previous code still has an issue: if you set v.x = 10 for example, and the do a lookup of 